In [ ]:
import json, os
import cv2
import torch
from multiprocessing import Pool, Manager
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
# from aquabyte.akpd import AKPD
# from aquabyte.template_matching import find_matches_and_homography
# from aquabyte.biomass_estimator import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
# from aquabyte.data_loader import KeypointsDataset, NormalizeCentered2D, ToTensor, BODY_PARTS
from torch.utils.data import Dataset, DataLoader
# from aquabyte.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
from PIL import Image


# from aquabyte.akpd_scorer import generate_confidence_score
import pandas as pd
from keras.models import load_model
import boto3

import numpy as np
import plotly.express as px
import time
from matplotlib import pyplot as plt

from collections import defaultdict
import datetime as dt
import json
import numpy as np
from sklearn.linear_model import LinearRegression
from collections import defaultdict



In [ ]:
# extract dataframe
s3_access_utils = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

pen_id = 5
group_id = str(pen_id)
start_date = '2019-06-18'
end_date = '2019-07-02'
name = 'kjeppevikholmen'


query = f"""
    SELECT * FROM (
      (SELECT * FROM prod.crop_annotation cas
      INNER JOIN prod.annotation_state pas on pas.id=cas.annotation_state_id
      WHERE cas.service_id = (SELECT ID FROM prod.service where name='BATI')
      AND cas.annotation_state_id = 3
      AND cas.pen_id={pen_id}) a
    RIGHT JOIN 
      (SELECT left_crop_url, estimated_weight_g, estimated_k_factor, estimated_length_mm, akpd_score FROM prod.biomass_computations
      WHERE prod.biomass_computations.captured_at >= '{start_date}' and prod.biomass_computations.captured_at <= '{end_date}'
      AND prod.biomass_computations.akpd_score > 0.0) bc 
    ON 
      (a.left_crop_url=bc.left_crop_url)
    ) x
    WHERE x.captured_at >= '{start_date}' and x.captured_at <= '{end_date}'
    AND x.pen_id = {pen_id}
    AND x.group_id = '{pen_id}';
"""

# query = f"""
# SELECT captured_at, left_crop_url, estimated_weight_g, estimated_k_factor, akpd_score, annotation, camera_metadata FROM prod.biomass_computations
# WHERE prod.biomass_computations.captured_at >= '{start_date}' and prod.biomass_computations.captured_at <= '{end_date}'
# AND prod.biomass_computations.akpd_score > 0.99
# """

df = rds_access_utils.extract_from_database(query)
df = df.sort_values('captured_at')
df = df[df.akpd_score > 0.0].copy(deep=True)
df.index = pd.to_datetime(df.captured_at)
df['hour'] = df.index.hour


In [ ]:
df.to_csv(f'/root/data/alok/biomass_estimation/playground/{name}_pen_id_{pen_id}_{start_date}_{end_date}.csv')
f = f'/root/data/alok/biomass_estimation/playground/{name}_pen_id_{pen_id}_{start_date}_{end_date}.csv'
s3_access_utils.s3_client.upload_file(f, 'aquabyte-images-adhoc', f'alok/production_datasets/{name}_pen_id_{pen_id}_{start_date}_{end_date}/annotation_dataset.csv')




In [ ]:
# extract dataframe
s3_access_utils = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

pen_id = 114
group_id = str(pen_id)
start_date = '2020-10-12'
end_date = '2020-10-13'
name = 'movikodden'


query = f"""
    
    SELECT captured_at, left_crop_url, estimated_weight_g, estimated_k_factor, estimated_length_mm, akpd_score FROM prod.biomass_computations
    WHERE prod.biomass_computations.captured_at >= '{start_date}' and prod.biomass_computations.captured_at <= '{end_date}'
    AND prod.biomass_computations.akpd_score > 0.9
    AND pen_id=114
""".format(start_date, end_date)

# query = f"""
# SELECT captured_at, left_crop_url, estimated_weight_g, estimated_k_factor, akpd_score, annotation, camera_metadata FROM prod.biomass_computations
# WHERE prod.biomass_computations.captured_at >= '{start_date}' and prod.biomass_computations.captured_at <= '{end_date}'
# AND prod.biomass_computations.akpd_score > 0.99
# """

df = rds_access_utils.extract_from_database(query)
df = df.sort_values('captured_at')
df = df[df.akpd_score > 0.99].copy(deep=True)
df.index = pd.to_datetime(df.captured_at)
df['hour'] = df.index.hour


In [ ]:
df['kf'] = 1e5 * df.estimated_weight_g.values / (df.estimated_length_mm.values ** 3)

In [ ]:
df.kf.mean()

In [ ]:
df.estimated_length_mm.mean()

In [ ]:
1e5 * df.estimated_weight_g.mean() / (df.estimated_length_mm.mean() ** 3)

In [ ]:
diffs = []
count = 0
for idx, row in df.iterrows():
    if count % 100 == 0:
        print(count)
    count += 1
    ann = row.annotation
    ann_left_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['leftCrop']}
    ann_right_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['rightCrop']}
    for bp in BODY_PARTS:
        diffs.append(ann_left_dict[bp][1] - ann_right_dict[bp][1])


In [ ]:
row.camera_metadata

In [ ]:
df.head()

In [ ]:
pen_id = 5
group_id = str(pen_id)
start_date = '2019-06-05'
end_date = '2020-07-02'
name = 'kjeppevikholmen'

In [ ]:
df.tail()

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(np.log(1 - df.akpd_score))
plt.grid()
plt.show()

In [ ]:
diffs = []
for idx, row in df[df.akpd_score > 0.95].iterrows():
    ann_c = row.annotation
    ann_dict_left_kps_c = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_c['leftCrop']}
    ann_dict_right_kps_c = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_c['rightCrop']}
    for bp in BODY_PARTS:
        diff = ann_dict_left_kps_c[bp][1] - ann_dict_right_kps_c[bp][1]
        diffs.append(diff)
print(np.mean(diffs))

In [ ]:
depths = []
for idx, row in df.iterrows():
    ann, cm = row.annotation, row.camera_metadata
    wkps = pixel2world(ann['leftCrop'], ann['rightCrop'], cm)
    depth = np.median([wkp[1] for wkp in wkps.values()])
    depths.append(depth)
    
df['depth'] = depths

In [ ]:
f_name = 'bolaks_pen_id_88_2020-02-10_2020-03-10.csv'
f = os.path.join('/root/data/alok/biomass_estimation/playground', f_name)
df.to_csv(f)
s3_access_utils.s3_client.upload_file(f, 'aquabyte-images-adhoc', 'alok/production_datasets/{}'.format(f_name))

In [ ]:
df.to_csv('/root/data/alok/biomass_estimation/playground/bolaks_data_full.csv')

In [ ]:
s3_access_utils.s3_client.upload_file('/root/data/alok/biomass_estimation/playground/bolaks_data_full.csv', 'aquabyte-images-adhoc', 'bolaks_data_full.csv')

<h1> Y-Coordinate Deviation Diagnosis </h1>

In [ ]:
df.to_hdf('/root/data/alok/biomass_estimation/playground/bolaks_data.h5', 'table')

In [ ]:
pd.read_hdf('/root/data/alok/biomass_estimation/playground/bolaks_data.h5', 'table')

In [ ]:
df = df.loc[:, ~df.columns.duplicated()]

In [ ]:
s3_access_utils.s3_client.upload_file('/root/data/alok/biomass_estimation/playground/bolaks_data.h5', 
                                      'aquabyte-images-adhoc', 
                                      'bolaks_data_full.h5')

In [ ]:
start_date = '2020-02-11'
FMT = '%Y-%m-%d'
dates = [dt.datetime.strftime(dt.datetime.strptime(start_date, FMT) + dt.timedelta(days=i), FMT) for i in range(27)]
for i in range(len(dates) - 1):
    start_date, end_date = dates[i], dates[i+1]
    diffs = []
    for idx, row in df[(df.captured_at > start_date) & (df.captured_at < end_date) & (df.akpd_score > 0.95)].iterrows():
        ann_c = row.annotation
        ann_dict_left_kps_c = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_c['leftCrop']}
        ann_dict_right_kps_c = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_c['rightCrop']}
        for bp in BODY_PARTS:
            diff = ann_dict_left_kps_c[bp][1] - ann_dict_right_kps_c[bp][1]
            diffs.append(diff)
    print('Mean y-coordinate difference on {}: {}'.format(start_date, np.mean(diffs)))

In [ ]:
mask = df.left_crop_url == 'https://aquabyte-crops.s3.eu-west-1.amazonaws.com/environment=production/site-id=56/pen-id=88/date=2020-02-11/hour=15/at=2020-02-11T15:24:13.622513000Z/left_frame_crop_1316_1714_3316_2715.jpg'
left_image_f, _, _ = s3_access_utils.download_from_url(df[mask].left_crop_url.iloc[0].iloc[0])
left_image = Image.open(left_image_f)
print(np.array(left_image).shape)
print(row.left_crop_metadata)

right_image_f, _, _ = s3_access_utils.download_from_url(df[mask].right_crop_url)
right_image = Image.open(right_image_f)
print(np.array(right_image).shape)
print(row.right_crop_metadata)

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

query = """
SELECT * FROM keypoint_annotations
WHERE pen_id=88
AND keypoints is not null
AND keypoints -> 'leftCrop' is not null
AND keypoints -> 'rightCrop' is not null
AND is_qa = FALSE;
"""

mdf = rds_access_utils.extract_from_database(query)

In [ ]:
diffs = []
for idx, row in mdf.iterrows():
    ann_c = row.keypoints
    ann_dict_left_kps_c = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_c['leftCrop']}
    ann_dict_right_kps_c = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_c['rightCrop']}
    for bp in BODY_PARTS:
        diff = ann_dict_left_kps_c[bp][1] - ann_dict_right_kps_c[bp][1]
        diffs.append(diff)
print(np.median(diffs))

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(diffs, bins=100)
plt.grid()
plt.show()

In [ ]:
growth_rates[-1]

In [ ]:
with open('/root/data/alok/biomass_estimation/playground/bolaks_raw_data.csv', 'w') as f:
    for idx in range(len(adj_weights) - 1):
        f.write('{},\n'.format(adj_weights[idx]))
    f.write(str(adj_weights[-1]))


In [ ]:
raw_avg_weight, raw_sample_size, smart_average, metadata, adj_weights = compute_metrics('2020-03-06', records_json)

In [ ]:
day_mask = (df.hour > 7) & (df.hour < 16)
# low_kf_mask = df.k_factor < 1.0
tdf = df[day_mask]

# get daily averages and sample sizes

records = defaultdict(list)
for date in sorted(list(set(tdf.index.date.astype(str)))):
    records[date].extend(tdf[date].estimated_weight_g.values.tolist())

records_json = json.dumps(records)

dates = sorted(list(set(tdf.index.date.astype(str))))
raw_avg_weights, raw_sample_sizes, growth_rates, trend_scores, smart_averages, distribution_confidences = [], [], [], [], [], []
for date in dates:
    raw_avg_weight, raw_sample_size, smart_average, metadata = compute_metrics(date, records_json)
    growth_rates.append(smart_average['dailyGrowthRate'])
    trend_scores.append(metadata['trend_score'])
    raw_avg_weights.append(raw_avg_weight)
    raw_sample_sizes.append(raw_sample_size)
    smart_averages.append(smart_average['weightMovingAvg'])
    distribution_confidences.append(metadata['distribution_confidence'])


In [ ]:
fig, axes = plt.subplots(5, 1, figsize=(10, 20))
x_values = tdf.estimated_weight_g.resample('D').agg(lambda x: x.mean()).dropna().index
axes[0].plot(x_values, raw_avg_weights, label='Raw Avg.')
axes[0].plot(x_values, smart_averages, label='Smart Avg.')
axes[0].plot(x_values, 1.02 * np.array(smart_averages), color='red', linestyle='--', label='Smart Avg. +/-2%')
axes[0].plot(x_values, 0.98 * np.array(smart_averages), color='red', linestyle='--')
axes[1].plot(x_values, raw_sample_sizes, label='Raw Daily Sample Size')
axes[2].plot(x_values, growth_rates)
axes[3].plot(x_values, trend_scores)
axes[4].plot(x_values, distribution_confidences)
for i, title in zip([0, 1, 2, 3, 4], ['Avg. weight', 'Raw Sample Size', 'Growth rate', 'Local trend score', 'Distribution Instability']):
    axes[i].set_title(title)
    axes[i].grid()
    axes[i].legend()
plt.show()

In [ ]:
smart_average['weightMovingAvg']

In [ ]:
urls = list(df[(df.captured_at > '2020-03-01') & (df.captured_at < '2020-03-06') & (day_mask)].left_crop_url.values[:, 0])
base_keys = []
for url in urls:
    base_key = os.path.dirname(url[url.index('environment'):])
    if base_key not in base_keys:
        base_keys.append(base_key)


In [ ]:
import csv
out_f = '/root/data/alok/biomass_estimation/playground/bolaks_data.csv'
with open(out_f, 'w', newline='\n') as f:
    for line in base_keys[:-1]:
        f.write(line + ',\n')
    f.write(base_keys[-1])
f.close()

In [ ]:
growth_rates, trend_scores, raw_avg_weights, raw_sample_sizes, smart_averages = \
defaultdict(list), defaultdict(list), defaultdict(list), defaultdict(list), defaultdict(list)
for lo in [6, 7, 8]:
    for hi in [14, 15, 16, 17]:
        key = '{}-{}'.format(lo, hi)
        print(key)
        day_mask = (df.hour > lo) & (df.hour < hi)
        tdf = df[day_mask]

        # get daily averages and sample sizes

        records = defaultdict(list)
        for date in sorted(list(set(tdf.index.date.astype(str)))):
            records[date].extend(tdf[date].estimated_weight_g.values.tolist())

        records_json = json.dumps(records)

        dates = sorted(list(set(tdf.index.date.astype(str))))
        for date in dates:
            raw_avg_weight, raw_sample_size, smart_average, metadata = compute_metrics(date, records_json)
            growth_rates[key].append(smart_average['dailyGrowthRate'])
            trend_scores[key].append(metadata['trend_score'])
            raw_avg_weights[key].append(raw_avg_weight)
            raw_sample_sizes[key].append(raw_sample_size)
            smart_averages[key].append(smart_average['weightMovingAvg'])


In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()
for lo in [6, 7, 8]:
    for hi in [14, 15, 16, 17]:
        key = '{}-{}'.format(lo, hi)
        fig.add_trace(go.Scatter(x=x_values[1:], y=smart_averages[key][1:],
                            mode='lines',
                            name=key))


fig.show()

In [ ]:
mdf = pd.DataFrame(smart_averages, index=x_values).round(2)
mdf['max_variation_pct'] = (mdf.max(axis=1) - mdf.min(axis=1)) / mdf.mean(axis=1)

In [ ]:
mdf

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8))
axes[0].plot(x_values[1:], smart_averages, label='Smart Avg.')
axes[0].set_title('Growth trend for fish with KF > 1')
axes[0].grid()

axes[1].plot(x_values, smart_averages_2, label='Smart Avg. 2')
axes[1].set_title('Growth trend for fish with KF < 1')
axes[1].grid()
# axes[1].set_ylim([1400, 1700])

plt.show()


In [ ]:
np.log(smart_averages[-1] / smart_averages[0]) / len(smart_averages)

In [ ]:
np.log(smart_averages_2[-1] / smart_averages_2[0]) / len(smart_averages_2)

In [ ]:
growth_rates[-1]

In [ ]:
day_mask = (df.hour > 7) & (df.hour < 16)
low_kf_mask = df.k_factor < 1.0
tdf = df[~day_mask & low_kf_mask]

# get daily averages and sample sizes

records = defaultdict(list)
for date in sorted(list(set(tdf.index.date.astype(str)))):
    records[date].extend(tdf[date].estimated_weight_g.values.tolist())

records_json = json.dumps(records)

dates = sorted(list(set(tdf.index.date.astype(str))))
raw_avg_weights, raw_sample_sizes, growth_rates, trend_scores, smart_averages_2, distribution_confidences = [], [], [], [], [], []
for date in dates:
    raw_avg_weight, raw_sample_size, smart_average, metadata = compute_metrics(date, records_json)
    growth_rates.append(smart_average['dailyGrowthRate'])
    trend_scores.append(metadata['trend_score'])
    raw_avg_weights.append(raw_avg_weight)
    raw_sample_sizes.append(raw_sample_size)
    smart_averages_2.append(smart_average['weightMovingAvg'])
    distribution_confidences.append(metadata['distribution_confidence'])


In [ ]:
fig, axes = plt.subplots(5, 1, figsize=(10, 20))
x_values = tdf.estimated_weight_g.resample('D').agg(lambda x: x.mean()).dropna().index
axes[0].plot(x_values, raw_avg_weights, label='Raw Avg.')
axes[0].plot(x_values, smart_averages, label='Smart Avg.')
axes[0].plot(x_values, 1.02 * np.array(smart_averages), color='red', linestyle='--', label='Smart Avg. +/-2%')
axes[0].plot(x_values, 0.98 * np.array(smart_averages), color='red', linestyle='--')
axes[1].plot(x_values, raw_sample_sizes, label='Raw Daily Sample Size')
axes[2].plot(x_values, growth_rates)
axes[3].plot(x_values, trend_scores)
axes[4].plot(x_values, distribution_confidences)
for i, title in zip([0, 1, 2, 3, 4], ['Avg. weight', 'Raw Sample Size', 'Growth rate', 'Local trend score', 'Distribution Instability']):
    axes[i].set_title(title)
    axes[i].grid()
    axes[i].legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
hour_mask = (df.hour > 7) & (df.hour < 19)
plt.hist(df[~hour_mask & (df.captured_at > '2020-02-16') & (df.captured_at < '2020-02-17')].estimated_weight_g, bins=20)
plt.title('Aggregate Weight Distribution (24 hour data)')
plt.xlabel('Weight Prediction')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(df.estimated_weight_g, bins=50)
plt.title('Aggregate Weight Distribution (Daylight hours only)')
plt.xlabel('Weight Prediction')
plt.ylabel('Count')
plt.show()

In [ ]:
dates = sorted(list(set(df.index.date.astype(str))))
N = 2
for idx in range(len(dates)-N-1):
    start_date, end_date, date = dates[idx], dates[idx+N], dates[idx+N+1]
    mean_adjustment = df[date].estimated_weight_g.mean() - df[start_date:end_date].estimated_weight_g.mean()
    x = np.percentile(df[start_date:end_date].estimated_weight_g + mean_adjustment, list(range(100)))
    y = np.percentile(df[date].estimated_weight_g, list(range(100)))
    fig, axes = plt.subplots(1, 2, figsize=(10, 3))
    axes[0].scatter(x[1:99], y[1:99])
    axes[0].plot([x[1], x[99]], [x[1], x[99]], color='red')
    axes[0].set_title(date)
    axes[0].grid()
    
    lower_bound = int(df[start_date:date].estimated_weight_g.min() * 0.8)
    upper_bound = int(df[start_date:date].estimated_weight_g.max() * 1.2)
    
    axes[1].hist(df[start_date:end_date].estimated_weight_g, bins=list(np.arange(lower_bound, upper_bound, 100)), color='blue', alpha=0.5, density=True)
    axes[1].hist(df[date].estimated_weight_g, bins=list(np.arange(lower_bound, upper_bound, 100)), color='red', alpha=0.5, density=True)
    axes[1].set_title(date)
    axes[1].grid()
    
    plt.show()

In [ ]:
dates = sorted(list(set(df.index.date.astype(str))))
cols = 3
rows = len(dates) // 3
fig, axes = plt.subplots(rows, cols, figsize=(12, 8))
fig.tight_layout(pad=3.0)
for idx, date in enumerate(dates):
    if idx == 8:
        continue
    tdf = df[date][df[date].estimated_weight_g > 3000].groupby('hour')['estimated_weight_g'].agg(lambda x: x.shape[0])
    
    row, col = idx // 3, idx % 3
    axes[row, col].plot(tdf.index, tdf.values)
    axes[row, col].set_xlabel('Hour')
    axes[row, col].set_ylabel('Avg. Weight')
    axes[row, col].set_title(date)
    
plt.show()
    

In [ ]:
annotation_dataset_url = 'https://aquabyte-images-adhoc.s3-eu-west-1.amazonaws.com/alok/production_datasets/kjeppevikholmen_pen_5_2019-06-05_2019-07-03/annotation_dataset.csv'
annotation_dataset_f, _, _ = s3_access_utils.download_from_url(annotation_dataset_url)

In [ ]:
df = pd.read_csv(annotation_dataset_f)

<h1> Add in Length / K-Factor Analysis </h1>

In [ ]:
class KeypointsDataset(Dataset):
    """Keypoints dataset
    This is the base version of the dataset that is used to map 3D keypoints to a
    biomass estimate. The label is the weight, and the input is the 3D workd keypoints
    obtained during triangulation
    """

    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform


    def __len__(self):
        return self.df.shape[0]


    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        if self.transform:
            input_sample = {
                'keypoints': row.keypoints,
                'cm': row.camera_metadata,
                'stereo_pair_id': row.id,
            }
            if 'length' in dict(row).keys():
                input_sample['label'] = row.length
            sample = self.transform(input_sample)
            return sample

        world_keypoints = row.world_keypoints
        length = row.length

        sample = {'kp_input': world_keypoints, 'label': length, 'stereo_pair_id': row.id}

        return sample
    
class NormalizedStabilityTransform(object):
    """
        Transforms world keypoints into a more stable coordinate system - this will lead to better
        training / convergene
    """
    
    def __call__(self, sample):
        modified_kps, label, stereo_pair_id, cm = \
            sample['modified_kps'], sample['label'], sample['stereo_pair_id'], sample['cm']
        modified_wkps = pixel2world(modified_kps['leftCrop'], modified_kps['rightCrop'], cm)
        stabilized_coordinates = {}
        for bp in BODY_PARTS:
            wkp = modified_wkps[bp]
            stabilized_kp_info = [0.5 * wkp[0]/wkp[1], 0.5 * wkp[2]/wkp[1], 0.5 * 0.1/wkp[1]]
            stabilized_coordinates[bp] = stabilized_kp_info
            
        normalized_label = label
        
        transformed_sample = {
            'kp_input': stabilized_coordinates,
            'label': normalized_label,
            'stereo_pair_id': stereo_pair_id,
            'single_point_inference': sample.get('single_point_inference')
        }
        
        return transformed_sample
    
# TODO: Define your network architecture here
import torch
from torch import nn

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(24, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.output(x)
        return x
        

        
        
class NormalizeCentered2D(object):
    
    """
    Transforms the 2D left and right keypoints such that:
        (1) The center of the left image 2D keypoints is located at the center of the left image
            (i.e. 2D translation)
        (2) The left image keypoints are possibly flipped such that the upper-lip x-coordinate 
            is greater than the tail-notch coordinate. This is done to reduce the total number of 
            spatial orientations the network must learn from -> reduces the training size
        (3) The left image keypoints are then rotated such that upper-lip is located on the x-axis.
            As in (2), this is done to reduce the total number of spatial orientations the network 
            must learn from -> reduces the training size
        (4) Rescale all left image keypoints by some random number between 'lo' and 'hi' args
        (5) Apply Gaussian random noise "jitter" to each keypoint to mimic annotation error
        (5) For all transformations above, the right image keypoint coordinates are accordingly
            transformed such that the original disparity values are preserved for all keypoints
            (or adjusted during rescaling event)
    """


    def flip_center_kps(self, left_kps, right_kps):

        x_min_l = min([kp[0] for kp in left_kps.values()])
        x_max_l = max([kp[0] for kp in left_kps.values()])
        x_mid_l = np.mean([x_min_l, x_max_l])

        y_min_l = min([kp[1] for kp in left_kps.values()])
        y_max_l = max([kp[1] for kp in left_kps.values()])
        y_mid_l = np.mean([y_min_l, y_max_l])

        x_min_r = min([kp[0] for kp in right_kps.values()])
        x_max_r = max([kp[0] for kp in right_kps.values()])
        x_mid_r = np.mean([x_min_r, x_max_r])

        y_min_r = min([kp[1] for kp in right_kps.values()])
        y_max_r = max([kp[1] for kp in right_kps.values()])
        y_mid_r = np.mean([y_min_r, y_max_r])

        fc_left_kps, fc_right_kps = {}, {}
        flip_factor = 1 if left_kps['UPPER_LIP'][0] > left_kps['TAIL_NOTCH'][0] else -1
        for bp in BODY_PARTS:
            left_kp, right_kp = left_kps[bp], right_kps[bp]
            if flip_factor > 0:
                fc_left_kp = np.array([left_kp[0] - x_mid_l, left_kp[1] - y_mid_l])
                fc_right_kp = np.array([right_kp[0] - x_mid_l, right_kp[1] - y_mid_l])
            else:
                fc_right_kp = np.array([x_mid_r - left_kp[0], left_kp[1] - y_mid_r])
                fc_left_kp = np.array([x_mid_r - right_kp[0], right_kp[1] - y_mid_r])
            fc_left_kps[bp] = fc_left_kp
            fc_right_kps[bp] = fc_right_kp

        return fc_left_kps, fc_right_kps


    def _rotate_cc(self, p, theta):
        R = np.array([
            [np.cos(theta), -np.sin(theta)],
            [np.sin(theta), np.cos(theta)]
        ])

        rotated_kp = np.dot(R, p)
        return rotated_kp


    def rotate_kps(self, left_kps, right_kps):
        upper_lip_x, upper_lip_y = left_kps['UPPER_LIP']
        theta = np.arctan(upper_lip_y / upper_lip_x)
        r_left_kps, r_right_kps = {}, {}
        for bp in BODY_PARTS:
            rotated_kp = self._rotate_cc(left_kps[bp], -theta)
            r_left_kps[bp] = rotated_kp
            disp = abs(left_kps[bp][0] - right_kps[bp][0])
            r_right_kps[bp] = np.array([rotated_kp[0] - disp, rotated_kp[1]])

        return r_left_kps, r_right_kps


    def scale_kps(self, left_kps, right_kps, factor):
        s_left_kps, s_right_kps = {}, {}
        for bp in BODY_PARTS:
            left_kp, right_kp = left_kps[bp], right_kps[bp]
            s_left_kps[bp] = factor * np.array(left_kps[bp])
            s_right_kps[bp] = factor * np.array(right_kps[bp])

        return s_left_kps, s_right_kps


    def jitter_kps(self, left_kps, right_kps, jitter):
        j_left_kps, j_right_kps = {}, {}
        for bp in BODY_PARTS:
            j_left_kps[bp] = np.array([left_kps[bp][0] + np.random.normal(0, jitter), 
                                       left_kps[bp][1] + np.random.normal(0, jitter)])
            j_right_kps[bp] = np.array([right_kps[bp][0] + np.random.normal(0, jitter), 
                                        right_kps[bp][1] + np.random.normal(0, jitter)])

        return j_left_kps, j_right_kps



    def modify_kps(self, left_kps, right_kps, factor, jitter, cm, rotate=True, center=False):
        fc_left_kps, fc_right_kps = self.flip_center_kps(left_kps, right_kps)
        if rotate:
            r_left_kps, r_right_kps = self.rotate_kps(fc_left_kps, fc_right_kps)
            s_left_kps, s_right_kps = self.scale_kps(r_left_kps, r_right_kps, factor)
        else:
            s_left_kps, s_right_kps = self.scale_kps(fc_left_kps, fc_right_kps, factor)
        j_left_kps, j_right_kps  = self.jitter_kps(s_left_kps, s_right_kps, jitter)
        j_left_kps_list, j_right_kps_list = [], []
        if not center:
            for bp in BODY_PARTS:
                l_item = {
                    'keypointType': bp,
                    'xFrame': j_left_kps[bp][0] + cm['pixelCountWidth'] / 2.0,
                    'yFrame': j_left_kps[bp][1] + cm['pixelCountHeight'] / 2.0
                }

                r_item = {
                    'keypointType': bp,
                    'xFrame': j_right_kps[bp][0] + cm['pixelCountWidth'] / 2.0,
                    'yFrame': j_right_kps[bp][1] + cm['pixelCountHeight'] / 2.0
                }

                j_left_kps_list.append(l_item)
                j_right_kps_list.append(r_item)
        else:
            for bp in BODY_PARTS:
                l_item = {
                    'keypointType': bp,
                    'xFrame': j_left_kps[bp][0],
                    'yFrame': j_left_kps[bp][1]
                }

                r_item = {
                    'keypointType': bp,
                    'xFrame': j_right_kps[bp][0],
                    'yFrame': j_right_kps[bp][1]
                }

                j_left_kps_list.append(l_item)
                j_right_kps_list.append(r_item)


        modified_kps = {
            'leftCrop': j_left_kps_list,
            'rightCrop': j_right_kps_list
        }

        return modified_kps

    
    def __init__(self, lo=None, hi=None, jitter=0.0, rotate=True, center=False):
        self.lo = lo
        self.hi = hi
        self.jitter = jitter
        self.rotate = rotate
        self.center = center
    

    def __call__(self, sample):
        keypoints, cm, stereo_pair_id, label = \
            sample['keypoints'], sample['cm'], sample.get('stereo_pair_id'), sample.get('label')
        left_keypoints_list = keypoints['leftCrop']
        right_keypoints_list = keypoints['rightCrop']
        left_kps = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in left_keypoints_list}
        right_kps = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in right_keypoints_list}
        
        factor = 1.0 
        if self.lo and self.hi:
            factor = np.random.uniform(low=self.lo, high=self.hi)
            
        jitter = np.random.uniform(high=self.jitter)
        
        modified_kps = self.modify_kps(left_kps, right_kps, factor, jitter, cm, 
            rotate=self.rotate, center=self.center)

        kp_input = {}
        for idx, _ in enumerate(modified_kps['leftCrop']):
            left_item, right_item = modified_kps['leftCrop'][idx], modified_kps['rightCrop'][idx]
            bp = left_item['keypointType']
            kp_input[bp] = [left_item['xFrame'], left_item['yFrame'], right_item['xFrame'], right_item['yFrame']]


        transformed_sample = {
            'kp_input': kp_input,
            'modified_kps': modified_kps,
            'label': label,
            'stereo_pair_id': stereo_pair_id,
            'cm': cm,
            'single_point_inference': sample.get('single_point_inference')
        }
        
        return transformed_sample
        

class ToTensor(object):
    
    def __call__(self, sample):
        kp_input, label, stereo_pair_id = \
            sample['kp_input'], sample.get('label'), sample.get('stereo_pair_id')
        
        x = []
        for bp in BODY_PARTS:
            kp_data = kp_input[bp]
            x.append(kp_data)
        if sample.get('single_point_inference'):
            x = np.array([x])
        else:
            x = np.array(x)
        
        kp_input_tensor = torch.from_numpy(x).float()
        
        tensorized_sample = {
            'kp_input': kp_input_tensor
        }

        if label:
            label_tensor = torch.from_numpy(np.array([label])).float() if label else None
            tensorized_sample['label'] = label_tensor

        if stereo_pair_id:
            tensorized_sample['stereo_pair_id'] = stereo_pair_id


        
        return tensorized_sample


In [ ]:
model_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/trained_models/2020-06-05T100500/kf_predictor.pb'
model_f, _, _ = s3_access_utils.download_from_url(model_url)

network = torch.load(model_f)
normalize_centered_2D_transform = NormalizeCentered2D()
normalized_stability_transform = NormalizedStabilityTransform()
to_tensor_transform = ToTensor()

In [ ]:
def generate_kf(row_id, akpd_keypoints, cm):

    # run length estimation
    input_sample = {
        'keypoints': akpd_keypoints,
        'cm': cm,
        'stereo_pair_id': row_id,
        'single_point_inference': True
    }
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform.__call__(input_sample)

    normalized_stability_kps = normalized_stability_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(normalized_stability_kps)
    kf_prediction = network(tensorized_kps['kp_input']).item()
    
    return kf_prediction


In [ ]:
kfs = []
args = []
count = 0
for idx, row in df.iterrows():
    cm = row.camera_metadata
    akpd_keypoints = row.annotation
    row_id = idx
    kf = generate_kf(row_id, akpd_keypoints, cm)
    kfs.append(kf)
    
    if count % 100 == 0:
        print(count)
    count += 1

In [ ]:
df['estimated_k_factor'] = kfs



In [ ]:
import numpy as np

In [ ]:
BODY_PARTS = sorted([
    'ADIPOSE_FIN',
    'ANAL_FIN',
    'DORSAL_FIN',
    'EYE',
    'PECTORAL_FIN',
    'PELVIC_FIN',
    'TAIL_NOTCH',
    'UPPER_LIP'
])

In [ ]:
import numpy as np

In [ ]:
from research.weight_estimation.keypoint_utils.optics import pixel2world

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
mask = (df.captured_at > '2020-08-22') & (df.estimated_weight_g < 1120)


In [ ]:
plt.figure(figsize=(20, 10))
plt.grid()
plt.hist(df[mask].estimated_weight_g)
plt.show()